# Procesamiento del Conjunto Consolidado
A través del siguiente notebook se realiza el procesamiento inicial para el análisis del conjunto de datos, se va a realizar lo siguiente:
- Descarte de columnas con información innecesaria
- Tratamiento inicial de las columnas financieras
- Tratamiento de las variables educativas
- Tratamiento de las variables condicionadas

In [83]:
# Carga de librerias
import pandas as pd
import os

In [84]:
# Carga del conjunto de datos consolidado
path= r"C:\Users\andre\OneDrive\Escritorio\Proyecto de Grado\result\db_consolidado_mensual"
month= 'Enero'
ruta_carga= os.path.join(path, month + '.csv')
data= pd.read_csv(ruta_carga, sep= ';', encoding= 'latin-1')

# Selección del conjunto de datos en la población objetivo
df= data[(data['EDAD'].isin(range(18,29))) & (data['ACTIVIDAD_OCUPADA_ULTIMA_SEMANA'].isin([1,2]))].reset_index(drop=True)

In [85]:
display(df.shape)
display(df.info())

(6902, 70)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6902 entries, 0 to 6901
Data columns (total 70 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   MES                                   6902 non-null   int64  
 1   DIRECTORIO                            6902 non-null   int64  
 2   SECUENCIA_P                           6902 non-null   int64  
 3   ORDEN                                 6902 non-null   int64  
 4   AREA_METROPOLITANA                    5203 non-null   float64
 5   ZONA_TERRITORIAL                      6902 non-null   int64  
 6   FACTOR_EXPANSION                      6902 non-null   float64
 7   DPTO                                  6902 non-null   int64  
 8   SEXO                                  6902 non-null   int64  
 9   EDAD                                  6902 non-null   int64  
 10  PARENTESCO_JEFE_DE_HOGAR              6902 non-null   int64  
 11  MADRE_RESIDE_HOGA

None

### Columnas a descartar

Se descartan aquellas columnas cuya información no es relevante sobre el objetivo del trabajo, o son aquellas que son condicionadas y la información que trae es poco relevante.

In [86]:

df = df.drop(columns=['AREA_METROPOLITANA', 'ALGUNA_VEZ_CAMPESINO',
                      'CONYUGE_VIVE_HOGAR', 'TIPO_REGIMEN_SALUD',
                      'QUIEN_PAGA_SALUD', 'VALOR_SALUD',
                      'INSTITUCION_PUBLICA_PRIVADA', 'USO_COMPARTIDO_SANITARIO',
                      'TIPO_COMBUSTIBLE_COCINAR', 'MAYOR_GRADO_ALCANZADO',
                      'TITULO_OBTENIDO'])

### Descarte columnas de identificación

In [87]:
df = df.drop(columns=['MES','SECUENCIA_P','ORDEN'])

### Revisión cuentas financieras
En las variables relacionadas con la posesión de algún elemento financiero presenta la particularidad qué cuando la respuesta es afirmativa se contesto con el valor '1', en el caso que no tuviera ese elemento el valor está vacío, así se va a reemplazar por 0 este valor nulo.

In [88]:
# Identificación de las variables financieras
financieros = ['HOGAR_TIENE_CUENTA_CORRIENTE',
'HOGAR_TIENE_CUENTA_AHORROS',
'HOGAR_TIENE_CDT',
'HOGAR_TIENE_PRESTAMO_COMPRA_VIVIENDA',
'HOGAR_TIENE_PRESTAMO_COMPRA_VEHICULO',
'HOGAR_TIENE_PRESTAMO_LIBRE_INVERSION',
'HOGAR_TIENE_TARJETA_CREDITO',
'HOGAR_TIENE_NINGUNO_DE_ESTOS',
'HOGAR_NO_SABE_FINANCIERO',
'HOGAR_TIENE_OTRO_FINANCIERO']

In [89]:
# Se rellenan los datos vacios con el valor de 0 que es el valor buscado.
df[financieros] = df[financieros].fillna(0)

In [90]:
# Se verifica que en todos los casos la suma es correcta y no hay datos nulos
df[financieros].sum(axis=1).value_counts()

1.0    6015
2.0     675
3.0     182
4.0      24
5.0       5
7.0       1
Name: count, dtype: int64

### Tratamiento de las variables condicionadas
Hay cierta cantidad de variables condicionadas que si son necesarias para el objetivo del estudio como el estrato socioeconómico, en el caso de los valores nulos en esta variable dado que no debían hacerlo, se va a añadir un valor adicional como el caso de "No tiene servicio eléctrico".

In [91]:
df['ESTRATO_ENERGIA_ELECTRICA'].value_counts()

ESTRATO_ENERGIA_ELECTRICA
1.0    2808
2.0    2333
3.0    1103
4.0     300
0.0     150
5.0      96
9.0      63
6.0      23
Name: count, dtype: int64

In [92]:
df['ESTRATO_ENERGIA_ELECTRICA'] = df['ESTRATO_ENERGIA_ELECTRICA'].fillna(10)

In [93]:
df['ESTRATO_ENERGIA_ELECTRICA'].value_counts()

ESTRATO_ENERGIA_ELECTRICA
1.0     2808
2.0     2333
3.0     1103
4.0      300
0.0      150
5.0       96
9.0       63
10.0      26
6.0       23
Name: count, dtype: int64

## Renombramiento de Palabras
Se va a realizar el cambio de los valores categóricos numéricos a su valor real como cadena de texto para profundizar en su análisis.

In [94]:
# Función que se realiza el cambio necesario
def replace_values(df, column, mapping):
    df[column] = df[column].replace(mapping)
    return df

In [95]:
## Cambios de valor codificado Original
cambios ={
'ZONA_TERRITORIAL':{
    1:'Cabecera',
    2:'Resto'
    },
'DPTO':{
    5: 'ANTIOQUIA',
    8: 'ATLANTICO',
         11: 'BOGOTA',
         13: 'BOLIVAR',
         15: 'BOYACA',
         17: 'CALDAS',
         18: 'CAQUETA',
         19: 'CAUCA',
         20: 'CESAR',
         23: 'CORDOBA',
         25: 'CUNDINAMARCA',
         27: 'CHOCO',
         41: 'HUILA',
         44: 'LA GUAJIRA',
         47: 'MAGDALENA',
         50: 'META',
         52: 'NARIÑO',
         54: 'NORTE DE SANTANDER',
         63: 'QUINDIO',
         66: 'RISALRALDA',
         68: 'SANTANDER',
         70: 'SUCRE',
         73: 'TOLIMA',
         76: 'VALLE',
         81: 'ARAUCA',
         85: 'CASANARE',
         86: 'PUTUMAYO',
         88: 'SAN ANDRES',
         91: 'AMAZONAS',
         94: 'GUAINIA',
         95: 'GUAVIARE',
         97: 'VAUPES',
         99: 'VICHADA'},
'SEXO':{
    1:'Masculino',
    2:'Femenino'
        },
'PARENTESCO_JEFE_DE_HOGAR':{
    1:'Jefe (a) del hogar',
    2:'Pareja, esposo(a), cónyuge, compañero(a)',
    3:'Hijo(a), hijastro(a)',
    4:'Padre o madre',
    5:'Suegro(a)',
    6:'Hermano(a) o hermanastro(a)',
    7:'Yerno o nuera',
    8:'Nieto(a)',
    9:'Otro  pariente',
    10:'Empleado(a) del servicio doméstico y sus parientes',
    11:'Pensionista',
    12:'Trabajador',
    13:'Otro no pariente'
    },
'MADRE_RESIDE_HOGAR':{
    1:'Si',
    2:'No',
    3:'Fallecida'
    },
'PADRE_RESIDE_HOGAR':{
    1:'Si',
    2:'No',
    3:'Fallecido'
    },
'SE_CONSIDERA_CAMPESINO':{
    1:'Si',
    2:'No',
    9:'No informa'
    },
'COMUNIDAD_ES_CAMPESINA':{
    1:'Si',
    2:'No',
    9:'No informa'
    },
'ETNIA':{
    1:'Indígena',
    2:'Gitano (Rom)',
    3:'Raizal del archipiélago',
    4:'Palenquero',
    5:'Negro, mulato (afrodescendiente)',
    6:'Ninguno de los anteriores'
    },
'ESTADO_CIVIL':{
    1:'No esta casado(a) y vive en pareja  hace  menos de dos años',
    2:'No esta  casado (a)  y vive en pareja   hace dos años  o más',
    3:'Esta casado (a)',
    4:'Esta separado (a) o divorciado (a)',
    5:'Esta viudo (a)',
    6:'Esta soltero (a)'
    },
'AFILIADO_SALUD':{
    1:'Si',
    2:'No',
    9:'No sabe, no informa'
    },
'DIFICULTAD_OIR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_HABLAR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_VER':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_MOVER_SUBIR_BAJAR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_AGARRAR':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_ENTENDER':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_BANARSE_VESTIRSE':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'DIFICULTAD_RELACIONARSE':{
    1:'No puede hacerlo',
    2:'Sí, con mucha dificultad',
    3:'Sí, con alguna dificultad',
    4:'Sin dificultad'
    },
'LEER_ESCRIBIR':{
    1:'Si',
    2:'No'
    },
'ACTUALMENTE_ESTUDIA':{
    1:'Si',
    2:'No'
    },
'MAYOR_NIVEL_EDUCATIVO':{
    1:'Ninguno',
    2:'Preescolar ',
    3:'Básica primaria (1o - 5o)',
    4:'Básica secundaria (6o - 9o)',
    5:'Media académica (Bachillerato clásico)',
    6:'Media técnica (Bachillerato técnico)',
    7:'Normalista',
    8:'Técnica profesional',
    9:'Tecnológica ',
    10:'Universitaria',
    11:'Especialización ',
    12:'Maestría ',
    13:'Doctorado ',
    99:'No sabe, no informa'
},
'ORIENTACION_SEXUAL':{
    1:'Hombres',
    2:'Mujeres',
    3:'Ambos sexos',
    4:'Otro'
    },
'GENERO':{
    1:'Hombre',
    2:'Mujer',
    3:'Hombre trans',
    4:'Mujer trans',
    5:'Otro'
    },
'ACTIVIDAD_OCUPADA_ULTIMA_SEMANA':{
    1:1,
    2:0
    },
'TIPO_VIVIENDA':{
    1:'Casa',
    2:'Apartamento',
    3:'Cuarto (s) en inquilinato',
    4:'Cuarto (s) en otro  tipo de estructura',
    5:'Vivienda indígena',
    6:'Otra vivienda (carpa,  vagón, embarcación, cueva, refugio natural, etc.)'
    },
'MATERIAL_PAREDES':{
    1:'Ladrillo, bloque, material prefabricado, piedra',
    2:'Madera pulida',
    3:'Adobe o tapia pisada',
    4:'Bahareque',
    5:'Madera burda, tabla, tablón',
    6:'Guadua',
    7:'Caña, esterilla, otro tipo de material vegetal',
    8:'Zinc, tela, cartón, latas, desechos, plástico',
    9:'Sin paredes'
    },
'MATERIAL_PISOS':{
    1:'Tierra, arena',
    2:'Cemento, gravilla',
    3:'Madera burda, tabla, tablón, otro vegetal',
    4:'Baldosín, ladrillo, vinisol, otros materiales sintéticos',
    5:'Mármol',
    6:'Madera pulida',
    7:'Alfombra o tapete de pared a pared'
    },
'SERVICIOS_ENERGIA_ELECTRICA':{
    1:'Si',
    2:'No'
    },
'ESTRATO_ENERGIA_ELECTRICA':{
    0:'Conexión Pirata',
    1:'Bajo - Bajo',
    2:'Bajo',
    3:'Medio - Bajo',
    4:'Medio',
    5:'Medio - Alto',
    6:'Alto',
    9:'No sabe o cuenta con Planta Eléctrica',
    10:'No tiene servicio eléctrico'
    },
'SERVICIOS_GAS_NATURAL':{
    1:'Si',
    2:'No'
    },
'SERVICIOS_ALCANTARILLADO':{
    1:'Si',
    2:'No'
    },
'SERVICIOS_RECOLECCION_BASURAS':{
    1:'Si',
    2:'No'
    },
'SERVICIOS_ACUEDUCTO':{
    1:'Si',
    2:'No'
    },
'TIPO_SANITARIO':{
    1:'Inodoro conectado a alcantarillado',
    2:'Inodoro conectado a pozo séptico',
    3:'Inodoro sin conexión',
    4:'Letrina',
    5:'Bajamar',
    6:'No tiene servicio sanitario'
    },
'COMO_ELIMINA_BASURA':{
    1:'Por recolección pública o privada',
    2:'La tiran a un río, quebrada, caño o laguna',
    3:'La tiran a un patio, lote, zanja o baldío',
    4:'La queman o entierran',
    5:'La eliminan de otra forma'
    },
'DONDE_OBTIENE_AGUA':{
    1:'De acueducto por tubería',
    2:'De otra fuente por tubería',
    3:'De pozo con bomba',
    4:'De pozo sin bomba, aljibe, jagüey o barreno',
    5:'Aguas lluvias',
    6:'Rio, quebrada, nacimiento o manantial ',
    7:'De pila pública',
    8:'Carrotanque',
    9:'Aguatero',
    10:'Agua embotellada o en bolsa'
    },
'DONDE_PREPARA_ALIMENTOS':{
    1:'En un cuarto usado sólo para cocinar?',
    2:'En un cuarto usado también para dormir?',
    3:'En una sala comedor con lavaplatos?',
    4:'En una sala comedor sin lavaplatos?',
    5:'En un patio, corredor, enramada, al aire libre?',
    6:'En ninguna parte, no preparan alimentos?'
    },
'TIPO_OCUPACION_VIVIENDA':{
    1:'Propia, totalmente pagada',
    2:'Propia, la están pagando',
    3:'En arriendo o subarriendo',
    4:'En usufructo',
    5:'Posesión sin título',
    6:'Propiedad colectiva',
    7:'Otra'
    },
'HOGAR_TIENE_CUENTA_CORRIENTE':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_CUENTA_AHORROS':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_CDT':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_PRESTAMO_COMPRA_VIVIENDA':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_PRESTAMO_COMPRA_VEHICULO':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_PRESTAMO_LIBRE_INVERSION':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_TARJETA_CREDITO':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_OTRO_FINANCIERO':{
    1:'Si',
    0:'No'
    },
'HOGAR_TIENE_NINGUNO_DE_ESTOS':{
    1:'Si',
    0:'No'
    },
'HOGAR_NO_SABE_FINANCIERO':{
    1:'Si',
    0:'No'
    },
}

In [96]:
for col in cambios.keys():
    df_pfinal = replace_values(df, col, cambios[col])

# Eliminación de simbolos especiales
Por medio de la libreria unidecode se van a eliminar los carácteres especiales.

In [97]:
from unidecode import unidecode

In [98]:
cat_variables = df_pfinal.drop(columns='ACTIVIDAD_OCUPADA_ULTIMA_SEMANA').select_dtypes(include=['object']).columns.to_list()
num_variables = df_pfinal.drop(columns='ACTIVIDAD_OCUPADA_ULTIMA_SEMANA').select_dtypes(include=['int64','float64']).columns.to_list()

In [99]:
for col in cat_variables:
    df[col] = df[col].apply(lambda x: unidecode(x))

In [100]:
path = r"C:\Users\andre\OneDrive\Escritorio\Proyecto de Grado\result\db_consolidado_procesado"
path_export = os.path.join(path, month +'_Procesado' +'.csv')
df_pfinal.to_csv(path_export, index=False, sep=';', encoding='latin1')